In [1]:
#need to pip install transformers
#import torch
#import loader
#import bert_encoder
import recommender
f1='ml-20m//movies.csv'
f2='ml-20m//ratings.csv'
f3='ml-20m//genome-scores.csv'
f4='ml-20m//genome-tags.csv'
f5='ml-20m//tags.csv'
data=recommender.movie_lens_data(f1,f2,f3,f4,f5, truncate=False)  #defaults to truncating movie data set. .movie_lens_data(f1,f2,f3,f4,f5,False) for entire data set  

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\Owner\Downloads\Rob\recommender.py:43: FutureWarning: The default value of regex will change from True to False in a future version. In addition, 

In [ ]:
data.bert_tokenize()

In [ ]:
recommendations = data.get_recommendations_by_title('Toy Story (1995)')
print(recommendations)

In [ ]:
user_keywords = "murder bloody"


recommendations = data.get_recommendations_by_keyword_bert_tfidf(user_keywords, num_recommendations=5, bert_weight=.5, tfidf_weight=.5)
print(recommendations)

recommendations = data.get_recommendations_by_keyword_bert_tfidf(user_keywords, num_recommendations=5, bert_weight=.5, tfidf_weight=.5, w_ratings=False)
print(recommendations)

After the above is executed, the following data frames are available

movie: contains 27278 unique movies with columns "movieID, title, genres"  (moviedID is unique for each row

ratings: contain 20,000,262 ratings with columns "userId movieId rating" the ratings range from .5-5. Each userID/movieId is unique

genome_scores: contain 11,709,767 unique rows with columns 'movieId, tagId, relevance'.  "relevance" represents the importance of the tag      to the movie, with higher scores indicating greater relevance.  movies with similar tag relevance scores may be more likely to be        recommended to a user who has rated one of those movies highly
genome_tags: 1127 different types of tags placed on a movie columns "tagId, tag".  Each row is unique

tags: 465,564 rows with columns "userId, movieId, tag, timestamp". The combination userId-movieId-tag is unique


In [ ]:
 # get the top N tags with the highest relevance scores
num_tags=10
tag_scores = data.genome_scores.groupby('tagId').agg({'relevance': 'mean'})
top_tags = tag_scores.sort_values('relevance', ascending=False)[:num_tags]

In [ ]:
data.movies


In [ ]:
#a movie can have multiple genres.  To put the genres of the 10 movie in the data frame in a list do this
genres = data.movies.loc[10, 'genres']
print(genres.split("|"))

Now encode data using the bert encoder. initialize the bert_encoder with a dataframe and then the column of the data frame you want to tokenize.
The bert_tokenize method is just using the movies data frame.  It concatenates the movie name and genres for the embeddings.
None of the other csv's are being used.

In [ ]:
selected_title = 'Guardian Angel (1994)'   #selected title must be of this specific format
suggestions = data.get_recommendations_by_title(selected_title)

In [ ]:
suggestions

In [ ]:
#import warnings
#warnings.filterwarnings("ignore", category=UserWarning)
#movie_list=data.movies['title'][:100].to_list()
import random
random.seed(42)
movie_list=random.sample(set(data.movies['title']), 350)
#movie_list = data.movies['title'].sample(n=350).tolist()

prec_list=[]
prec_avg=[]
epo=[]
epoch=0
for m in movie_list:
    print(m)
    epoch+=1
    suggestions = data.get_recommendations_by_title(m, num_recommendations=5)
    
    precision=data.get_evaluation_title(m, suggestions)
    #if precision!=0:
    prec_list.append(precision)
    precision_avg=sum(prec_list)/len(prec_list)
    prec_avg.append(precision_avg)
    epo.append(epoch)
import matplotlib.pyplot as plt
plt.plot(epo, prec_avg)
plt.xlabel('Epoch')
plt.ylabel('Avg_Precision')
plt.show()

In [ ]:
keyword_list=["Good comedy movies with Saturday Night Live cast members", "Horror movies that are really scary", 
              "Heartwarming drama", "famous opening theme", "dialogue driven storyline", "criminal mastermind",
              "dream sequence", "classic movies from the 90s",  "Steven Spielberg movies", "romantic comedies", "zombie movies",
              "French movies", "movie with a car chase", "movie set in the future", "movie with a box office success", 
              "movie that was critically acclaimed", "movie that was a critical and commercial success", "movie that you can watch over and over again",
              "movie with a happy ending", "movie with a lot of special effects", "good versus evil movie", "fictional biography", "movie based on short story",
              "corruption", "movie with flashback", "alternate history", "black comedy", "movies set in New York", "Oscar-winning movies", 
              "movies with Hans Zimmer soundtracks", "Marvel movies",  "feel-good movies", "Meryl Streep movies", "Star Wars movies", "musical movies",
              "movies set in space", "Cannes Film Festival winners", "movies with Tom Cruise", "murder bloody", "Animated movies", "Action-packed movies",
              "Documentaries", "Best movies of all time", "Cult favorites", "Must-see movies", "Mystery movies", "Social issue movies", "Time travel movies",
              "Dystopian movies", "Tearjerker movies", "Mystery movies", "Adrenaline-fueled movies", "Thought-provoking movies", "Visually stunning movies",
              "Holiday movies", "Sports movies", "Spy movies", "Mind-bending movies"]
prec_list=[]
prec_avg=[]
epo=[]
epoch=0
for m in keyword_list:
    print(m)
    epoch+=1
    suggestions = data.get_recommendations_by_keyword_bert_tfidf(m,num_recommendations=5,bert_weight=0.5,tfidf_weight=0.5,w_ratings=True)
    
    precision=data.get_evaluation_keyword(suggestions)
    #if precision!=0:
    prec_list.append(precision)
    precision_avg=sum(prec_list)/len(prec_list)
    prec_avg.append(precision_avg)
    epo.append(epoch)
import matplotlib.pyplot as plt
plt.plot(epo, prec_avg)
plt.xlabel('Epoch')
plt.ylabel('Avg_Precision')
plt.show()